In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [2]:
global pop_size
global parents
global epochs
global threshold
parents = []
pop_size = 40
epochs = 100
threshold = 0.735

In [3]:
df = pd.read_excel (r'Training_Data.xlsx')

In [4]:
df

,PatientId,EncounterId,DischargeDisposision,Gender,Race,DiabetesMellitus,ChronicKidneyDisease,Anemia,Depression,ChronicObstructivePulmonaryDisease,...,BetaBlockers,Diuretics,TotalMedicine,CardiacTroponin,Hemoglobin,SerumSodium,SerumCreatinine,BNP,NT-proBNP,ReadmissionWithin_90Days
0,4200412,199171333,Home,Male,White,DM,NaN,Anemia,NaN,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
1,4055894,26704337,Home,Male,White,DM,CKD,Anemia,Depression,COPD,...,1,5,8,0.0,0.00,0.0,1.540000,0.0,0.0,No
2,4867407,60388216,Home,Male,White,DM,CKD,Anemia,NaN,COPD,...,1,1,2,0.0,10.20,0.0,0.000000,0.0,0.0,No
3,4058064,274642265,Hospice - Home,Female,White,DM,NaN,Anemia,NaN,COPD,...,0,0,0,0.0,0.00,132.0,0.000000,0.0,0.0,No
4,4150623,70000001557327,SNF,Female,White,NaN,NaN,Anemia,NaN,COPD,...,0,0,0,0.0,7.26,0.0,0.000000,0.0,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,4152524,40004415567,Home Health,Female,White,DM,CKD,Anemia,NaN,COPD,...,2,3,6,0.0,0.00,0.0,1.076667,0.0,0.0,Yes
8477,4042227,14347947026,SNF,Male,White,DM,CKD,NaN,Depression,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
8478,4603405,67117733,Hospice,Female,White,NaN,NaN,NaN,NaN,NaN,...,1,0,1,0.0,0.00,0.0,0.000000,0.0,0.0,No
8479,4033677,68564389,Home,Female,White,DM,CKD,NaN,Depression,NaN,...,1,1,5,0.0,0.00,0.0,1.690000,0.0,0.0,Yes


# Preprocessing

In [5]:
for i in df:
    print(i, 'has', df[i].isnull().sum(), 'null values')

PatientId has 0 null values
EncounterId has 0 null values
DischargeDisposision has 0 null values
Gender has 0 null values
Race has 93 null values
DiabetesMellitus has 3857 null values
ChronicKidneyDisease has 3906 null values
Anemia has 3002 null values
Depression  has 5108 null values
ChronicObstructivePulmonaryDisease has 3954 null values
Age has 0 null values
ChronicDiseaseCount has 0 null values
LengthOfStay has 0 null values
EmergencyVisit has 0 null values
InpatientVisit has 0 null values
OutpatientVisit has 0 null values
TotalVisits has 0 null values
BMIMin has 0 null values
BMIMax has 0 null values
BMIMedian has 0 null values
BMIMean has 0 null values
BPDiastolicMin has 0 null values
BPDiastolicMax has 0 null values
BPDiastolicMedian has 0 null values
BPDiastolicMean has 0 null values
BPSystolicMin has 0 null values
BPSystolicMax has 0 null values
BPSystolicMedian has 0 null values
BPSystolicMean has 0 null values
TemperatureMin has 0 null values
TemperatureMax has 0 null value

In [6]:
def Preprocessing(df):
    df['Depression '] = df['Depression '].replace(to_replace = np.nan, value = "No Depression")
    df['Anemia'] = df['Anemia'].replace(to_replace = np.nan, value = "No Anemia")
    df['ChronicKidneyDisease'] = df['ChronicKidneyDisease'].replace(to_replace = np.nan, value = "No CKD")
    df['DiabetesMellitus'] = df['DiabetesMellitus'].replace(to_replace = np.nan, value = "No DM")
    df['ChronicObstructivePulmonaryDisease'] = df['ChronicObstructivePulmonaryDisease'].replace(to_replace = np.nan, value = "No COPD")
    df['Race'] = df['Race'].replace(to_replace = np.nan, value = df['Race'].mode()[0])
    df = df.astype({'EncounterId':'string'})
    
    return df

In [7]:
df = Preprocessing(df)

In [8]:
df

,PatientId,EncounterId,DischargeDisposision,Gender,Race,DiabetesMellitus,ChronicKidneyDisease,Anemia,Depression,ChronicObstructivePulmonaryDisease,...,BetaBlockers,Diuretics,TotalMedicine,CardiacTroponin,Hemoglobin,SerumSodium,SerumCreatinine,BNP,NT-proBNP,ReadmissionWithin_90Days
0,4200412,199171333,Home,Male,White,DM,No CKD,Anemia,No Depression,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
1,4055894,26704337,Home,Male,White,DM,CKD,Anemia,Depression,COPD,...,1,5,8,0.0,0.00,0.0,1.540000,0.0,0.0,No
2,4867407,60388216,Home,Male,White,DM,CKD,Anemia,No Depression,COPD,...,1,1,2,0.0,10.20,0.0,0.000000,0.0,0.0,No
3,4058064,274642265,Hospice - Home,Female,White,DM,No CKD,Anemia,No Depression,COPD,...,0,0,0,0.0,0.00,132.0,0.000000,0.0,0.0,No
4,4150623,70000001557327,SNF,Female,White,No DM,No CKD,Anemia,No Depression,COPD,...,0,0,0,0.0,7.26,0.0,0.000000,0.0,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,4152524,40004415567,Home Health,Female,White,DM,CKD,Anemia,No Depression,COPD,...,2,3,6,0.0,0.00,0.0,1.076667,0.0,0.0,Yes
8477,4042227,14347947026,SNF,Male,White,DM,CKD,No Anemia,Depression,COPD,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,Yes
8478,4603405,67117733,Hospice,Female,White,No DM,No CKD,No Anemia,No Depression,No COPD,...,1,0,1,0.0,0.00,0.0,0.000000,0.0,0.0,No
8479,4033677,68564389,Home,Female,White,DM,CKD,No Anemia,Depression,No COPD,...,1,1,5,0.0,0.00,0.0,1.690000,0.0,0.0,Yes


# Encoding

In [9]:
label_encoder = preprocessing.LabelEncoder()
df['Anemia'] = label_encoder.fit_transform(df['Anemia'])
df['DiabetesMellitus'] = label_encoder.fit_transform(df['DiabetesMellitus'])
df['ChronicKidneyDisease'] = label_encoder.fit_transform(df['ChronicKidneyDisease'])
df['Depression '] = label_encoder.fit_transform(df['Depression '])
df['ChronicObstructivePulmonaryDisease'] = label_encoder.fit_transform(df['ChronicObstructivePulmonaryDisease'])
df['DischargeDisposision'] = label_encoder.fit_transform(df['DischargeDisposision'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Race'] = label_encoder.fit_transform(df['Race'])
df['ReadmissionWithin_90Days'] = label_encoder.fit_transform(df['ReadmissionWithin_90Days'])
df['EncounterId'] = label_encoder.fit_transform(df['EncounterId'])

In [10]:
df

,PatientId,EncounterId,DischargeDisposision,Gender,Race,DiabetesMellitus,ChronicKidneyDisease,Anemia,Depression,ChronicObstructivePulmonaryDisease,...,BetaBlockers,Diuretics,TotalMedicine,CardiacTroponin,Hemoglobin,SerumSodium,SerumCreatinine,BNP,NT-proBNP,ReadmissionWithin_90Days
0,4200412,1286,7,1,6,0,1,0,1,0,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,1
1,4055894,1555,7,1,6,0,0,0,0,0,...,1,5,8,0.0,0.00,0.0,1.540000,0.0,0.0,0
2,4867407,5262,7,1,6,0,0,0,1,0,...,1,1,2,0.0,10.20,0.0,0.000000,0.0,0.0,0
3,4058064,2213,11,0,6,0,1,0,1,0,...,0,0,0,0.0,0.00,132.0,0.000000,0.0,0.0,0
4,4150623,6674,19,0,6,1,1,0,1,0,...,0,0,0,0.0,7.26,0.0,0.000000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,4152524,2950,8,0,6,0,0,0,1,0,...,2,3,6,0.0,0.00,0.0,1.076667,0.0,0.0,1
8477,4042227,455,19,1,6,0,0,1,0,0,...,0,0,0,0.0,0.00,0.0,0.000000,0.0,0.0,1
8478,4603405,5592,10,0,6,1,1,1,1,1,...,1,0,1,0.0,0.00,0.0,0.000000,0.0,0.0,0
8479,4033677,5857,7,0,6,0,0,1,0,1,...,1,1,5,0.0,0.00,0.0,1.690000,0.0,0.0,1


In [11]:
y = df['ReadmissionWithin_90Days']
x = df.drop('ReadmissionWithin_90Days', axis = 1)

# Chromosome Creation

In [12]:
chromosomes = []
while True:
    temp = x.sample(n = 30, axis = 'columns')
    temp = list(tuple(temp.columns))
    if(temp not in chromosomes):
        chromosomes.append(temp)
    if(len(chromosomes) == pop_size):
        break

In [13]:
def Fitness_function():
    acc = {}
    for i in range(pop_size):
        x_train, x_test, y_train, y_test = train_test_split(df[chromosomes[i]], y, test_size = 0.2, random_state = 42)
        clf = AdaBoostClassifier()
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        acc[i] = accuracy_score(y_test, pred)
    return acc

# Parent Selection 

In [14]:
def Create_population():
    acc = Fitness_function()
    while True:
        p1 = random.randint(0, 39)
        p2 = random.randint(0, 39)
        if(p1 != p2):
            if (acc[p1] > acc[p2]):
                if (chromosomes[p1] not in parents):
                    parents.append(chromosomes[p1])
            if (acc[p1] < acc[p2]):
                if (chromosomes[p2] not in parents):
                    parents.append(chromosomes[p2])       
        if(len(parents) == 20):
            break     

# Crossover of Parent chromosomes

In [15]:
def error_detection(child):
    keep = []
    remove = []
    idx = []
    n = 0
    for i in range(len(child)):
        if (child[i] not in keep):
             keep.append(child[i])
        else:
            remove.append(child[i])
            idx.append(i)
            
    for i in range(len(idx)):        
        while True:
            n = random.randint(0, (len(x.columns) - 1))
            col = list(x.columns)[n]
            if ((col not in keep) and (col not in remove)):
                child[idx[i]] = col
                keep.append(col)
                break
    return child

In [16]:
def Crossover():
    parent1 = parents[random.randint(0, 19)]
    parent2 = parents[random.randint(0, 19)]
    while (parent1 == parent2):
        parent2 = parents[random.randint(0, 19)]
        
    crossover_point = random.randint(1, 30)
    
    child1 = parent1[0:crossover_point] + parent2[crossover_point:]
    child2 = parent2[0:crossover_point] + parent1[crossover_point:]

    child1 = error_detection(child1)
    child2 = error_detection(child2)

    return child1, child2

# Mutation of Children

In [17]:
def Mutation(chromosome):
    chance = random.randint(0, 9)
    if (chance >= 4):
        while True:
            col = x.columns[random.randint(0, (len(x.columns) - 1))]
            if (col not in chromosome):
                index = random.randint(0, (len(chromosome) - 1))
                chromosome[index] = col
                break
    return chromosome

# Selection of chromosome with the best genes

In [18]:
break_condition = 0
Create_population()

for i in range(epochs):
    mutated_chromosomes = []
    for i in range(int(len(parents)/2)):
        c1, c2 = Crossover()
        m1 = Mutation(c1)
        m2 = Mutation(c2)
        mutated_chromosomes.append(m1)
        mutated_chromosomes.append(m2)
    new_gen = parents + mutated_chromosomes
    
    
    accu = []
    for i in range(len(new_gen)):
        x_train, x_test, y_train, y_test = train_test_split(df[new_gen[i]], y, test_size = 0.2, random_state = 42)
        clf = AdaBoostClassifier()
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        print(accuracy_score(y_test, pred))
        
        accu.append(accuracy_score(y_test, pred))
        if (accuracy_score(y_test, pred) > threshold):
            best_gene = new_gen[i]
            break_condition = 1
            break
            
    if (break_condition == 1):
        break     
        
    parents = []
    while True:
        p1 = random.randint(0, 39)
        p2 = random.randint(0, 39)
        if(p1 != p2):
            if (accu[p1] > accu[p2]):
                if (new_gen[p1] not in parents):
                    parents.append(chromosomes[p1])
            if (accu[p1] < accu[p2]):
                if (new_gen[p2] not in parents):
                    parents.append(chromosomes[p2])       
        if(len(parents) == 20):
            break  

0.7318797878609311
0.7259870359457867
0.7283441367118444
0.7212728344136712
0.7224513847967
0.7271655863288156
0.7118444313494402
0.7106658809664114
0.7259870359457867
0.7271655863288156
0.7177371832645846
0.723629935179729
0.714201532115498
0.7248084855627578
0.7147908073070124
0.72775486152033
0.72775486152033
0.7242192103712434
0.7200942840306424
0.7189157336476134
0.718326458456099
0.7318797878609311
0.7271655863288156
0.7136122569239834
0.7253977607542722
0.7224513847967
0.7348261638185032
0.7100766057748968
0.7153800824985268
0.708898055391868
0.7083087802003536
0.7301119622863878
0.7259870359457867
0.7342368886269888
0.7159693576900412
0.7024160282852092
0.7289334119033589
0.7224513847967
0.7224513847967
0.7112551561579258
0.7177371832645846
0.723629935179729
0.7212728344136712
0.7147908073070124
0.7224513847967
0.7124337065409546
0.7212728344136712
0.7212728344136712
0.7065409546258102
0.7212728344136712
0.7318797878609311
0.72775486152033
0.7248084855627578
0.7283441367118444


In [19]:
df[best_gene]

,BPSystolicMax,TemperatureMax,PulseRateMin,Gender,DischargeDisposision,TotalMedicine,ChronicObstructivePulmonaryDisease,Age,Depression,TemperatureMean,...,RespiratoryRateMax,SerumCreatinine,BPDiastolicMax,TemperatureMedian,BPDiastolicMean,OutpatientVisit,CardiacTroponin,RespiratoryRateMin,RespiratoryRateMedian,SerumSodium
0,0,0.00,0,1,7,0,0,58,1,0.000000,...,0,0.000000,0,0.00,0.000000,21,0.0,0,0.0,0.0
1,150,99.30,0,1,7,8,0,80,0,98.266667,...,22,1.540000,78,98.25,63.052632,0,0.0,14,17.5,0.0
2,0,0.00,0,1,7,2,0,63,1,0.000000,...,0,0.000000,0,0.00,0.000000,0,0.0,0,0.0,0.0
3,182,97.70,0,0,11,0,0,73,1,97.700000,...,22,0.000000,115,97.70,115.000000,0,0.0,22,22.0,132.0
4,159,98.60,71,0,19,0,0,85,1,97.854286,...,20,0.000000,72,97.88,59.044118,0,0.0,14,17.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,195,98.60,0,0,8,6,0,59,1,97.822222,...,107,1.076667,97,98.00,68.277778,0,0.0,9,26.0,0.0
8477,0,0.00,0,1,19,0,0,86,0,0.000000,...,0,0.000000,0,0.00,0.000000,0,0.0,0,0.0,0.0
8478,195,99.14,62,0,10,1,1,94,1,98.220000,...,22,0.000000,94,98.24,78.473684,0,0.0,1,16.0,0.0
8479,185,98.60,29,0,7,5,1,77,0,97.970000,...,18,1.690000,121,97.97,93.875000,0,0.0,16,17.0,0.0
